# Translite - A Low Resoure Modular Multi-Lingual NLP Translator

## Section 1 - Configuration
This is where the datasets will be prepared and configured. User is capable of adding as many languages as they'd like to this. The main requirement will be that datasets must be stored on Kaggle and in the following formats:
  - CSV
  - Excel
  - JSON
  - Parquet

This can be checked by looking at the **Data Explorer** on the Kaggle page for your dataset and seeing if the files have the extension ".json", ".csv", ".xlsx", or ".parquet".

#### 1.1 - Install Requirements

In [1]:
!pip install ipywidgets pandas kaggle

#### 1.2 - Load/Create Config

Here you will add your datasets for all of the languages you'd like to translate to English. You will need to add the url to the raw dataset, the name of the column where the English versions are, the name of the column where the non-English versions are stored, and the actual name of the language. It is suggested that you use the export button to save your config when you're done so that you can quickly import again when you return. A sample dataset with Spanish and Italian is available as well.

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from google.colab import files

df = pd.DataFrame(columns=[
    "Kaggle Dataset Name",
    "Language Name",
    "English Column Name",
    "Other Language Column Name"])

def display_table(df):
    clear_output(wait=True)
    display(ds_uri, lang, eng_col, natv_col)
    display(widgets.HBox((add_button, import_button, export_button, clear_button)))
    display(df)

def add_row(ds_uri, lang, eng_col, natv_col):
    global df
    new_row = pd.DataFrame([[ds_uri, lang, eng_col, natv_col]], columns=df.columns.tolist())
    df = pd.concat([df, new_row], ignore_index=True)
    display_table(df)

ds_uri = widgets.Text(description="Kaggle Dataset Name:",
                      layout=widgets.Layout(width="500px"),
                      style=dict(description_width='initial'))
lang = widgets.Text(description="Language Name:",
                      layout=widgets.Layout(width="500px"),
                      style=dict(description_width='initial'))
eng_col = widgets.Text(description="English Column Name:",
                      layout=widgets.Layout(width="500px"),
                      style=dict(description_width='initial'))
natv_col = widgets.Text(description="Other Language Column Name:",
                      layout=widgets.Layout(width="500px"),
                      style=dict(description_width='initial'))
add_button = widgets.Button(description="Add Row")
import_button = widgets.Button(description="Import Dataset")
export_button = widgets.Button(description="Export Dataset")
clear_button = widgets.Button(description="Clear Table")

def on_import_button_click(b):
    uploaded = files.upload()
    filename = next(iter(uploaded))
    global df
    df = pd.read_json(filename)
    display_table(df)

def on_export_button_click(b):
    df.to_json("translite_datasets.json", index=False)
    files.download("translite_datasets.json")

def on_add_button_click(b):
    add_row(ds_uri.value, lang.value, eng_col.value, natv_col.value)
    ds_uri.value = ""
    lang.value = ""
    eng_col.value = ""
    natv_col.value = ""

def on_clear_button_click(b):
    global df
    df = pd.DataFrame(columns=[
        "Kaggle Dataset Name",
        "Language Name",
        "English Column Name",
        "Other Language Column Name"])
    display_table(df)

add_button.on_click(on_add_button_click)
import_button.on_click(on_import_button_click)
export_button.on_click(on_export_button_click)
clear_button.on_click(on_clear_button_click)
display_table(df)


Text(value='', description='Kaggle Dataset Name:', layout=Layout(width='500px'), style=DescriptionStyle(descri…

Text(value='', description='Language Name:', layout=Layout(width='500px'), style=DescriptionStyle(description_…

Text(value='', description='English Column Name:', layout=Layout(width='500px'), style=DescriptionStyle(descri…

Text(value='', description='Other Language Column Name:', layout=Layout(width='500px'), style=DescriptionStyle…

,Kaggle Dataset Name,Language Name,English Column Name,Other Language Column Name


##### Load Sample (Optional)
If you just want to test this script and don't want to curate a list of languages yourself please use this feature to load a sample dataset.

In [3]:
import json
import pandas as pd
from google.colab import files

raw_config = '{"Kaggle Dataset Name":{"0":"devicharith\/language-translation-englishfrench","1":"lonnieqin\/englishspanish-translation-dataset"},"Language Name":{"0":"French","1":"Spanish"},"English Column Name":{"0":"English words\/sentences","1":"english"},"Other Language Column Name":{"0":"French words\/sentences","1":"spanish"}}'
df = pd.DataFrame(json.loads(raw_config))
df.to_json("translite_datasets.json", index=False)
files.download("translite_datasets.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### 1.3 - Connect to Kaggle API
Datasets for this product will need to be provided from Kaggle, this means that if you need to utilize cutom language translation datasets you must first host them on https://kaggle.com. You need to get a Kaggle API Key as well. To get this go to your account settings page on Kaggle, and click "Create New API Token". This will create a file called `kaggle.json` that you will upload here.



In [4]:
from google.colab import files
files.upload()
!mkdir -p ~/.config/kaggle
!cp kaggle.json ~/.config/kaggle/
!chmod 600 ~/.config/kaggle/kaggle.json

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

Saving kaggle.json to kaggle (2).json


#### 1.4 - Load Datasets from Config
This is where you'll test out your config and ensure that you can properly load datasets from it.

In [22]:
import os
import pandas as pd

from collections.abc import Callable as function

allowed_types: dict[str, function] = {
    'csv': pd.read_csv,
    'json': pd.read_json,
    'xlsx': pd.read_excel,
    'parquet': pd.read_parquet
}

id_counter: int = 0
dfs = []
classes: dict[int, str] = {}

for _, dataset in df.iterrows():
    print(f"Loading {dataset['Language Name']}")
    dataset_name: str = dataset["Kaggle Dataset Name"]
    language_name: str = dataset["Language Name"]

    # Get and prep datasets from kaggle
    !kaggle datasets download {dataset_name} -d {language_name}
    !unzip {dataset_name.split('/')[-1]}.zip -d {language_name}

    files: list[tuple[str, function]] = [(f, allowed_types.get(f.split('.')[-1]))
    for f in os.listdir(language_name)]

    curr = pd.DataFrame(columns=[
        "class", "english", "native"
    ])

    # Look for all files in the dir that are readable as datasets and
    # reformat them


    for f, fn in files:
      if fn:
        tmp = fn(f"{language_name}/{f}")
        tmp = tmp.rename(columns={
            dataset["English Column Name"]: "english",
            dataset["Other Language Column Name"]: "native"
        })
        tmp["class"] = language_name
        tmp["class_int"] = id_counter
        classes[f"{id_counter}"] = language_name
        curr = pd.concat([curr, tmp], ignore_index=True)
    id_counter += 1

    dfs.append(curr)

combined = pd.concat(dfs, ignore_index=True)
combined.to_csv("translite.csv", index=False)

Loading French
Dataset URL: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench
License(s): CC0-1.0
language-translation-englishfrench.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  language-translation-englishfrench.zip
replace French/eng_-french.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: French/eng_-french.csv  
Loading Spanish
Dataset URL: https://www.kaggle.com/datasets/lonnieqin/englishspanish-translation-dataset
License(s): unknown
englishspanish-translation-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  englishspanish-translation-dataset.zip
replace Spanish/data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Spanish/data.csv        


#### 1.5 - Prepare Datasets for Classification
This is where datasets will be prepared for classification model training.

##### 1.5.1 - Prepare Train Test Splits

In [23]:
TRAIN_SIZE = 0.85

print("Filling embedding reference map...")
combined = combined.sample(frac=1)
train_in = combined['native'][:int(len(combined) * TRAIN_SIZE)].tolist()
train_out = combined['class_int'][:int(len(combined) * TRAIN_SIZE)].tolist()
test_in = combined['native'][int(len(combined) * TRAIN_SIZE):].tolist()
test_out = combined['class_int'][int(len(combined) * TRAIN_SIZE):].tolist()

Filling embedding reference map...


## Section 2 - Models


#### 2.1 - Classifier Training

##### 2.1.1 - Training

In [24]:
# https://developer.ibm.com/tutorials/awb-classifying-data-multinomial-naive-bayes-algorithm/
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

model = Pipeline([
    ('embeddings', TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
    ('classifier', MultinomialNB(
        alpha=0.1))
])

model.fit(train_in, train_out)


Pipeline(steps=[('embeddings',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('classifier', MultinomialNB(alpha=0.1))])

##### 2.1.2 - Validation

In [25]:
predictions = model.predict(test_in)
acc = sum(predictions == test_out) / len(test_out)
print(f"Accuracy: {acc}")

Accuracy: 0.9983253371956187


##### 2.1.3 - Manual Test

In [44]:
print(classes)
print(classes[f"{int(model.predict([input('>>> ')]).item())}"])

{'0': 'French', '1': 'Spanish'}
>>> comme ca va
French


##### 2.1.4 - Export Classifier
Export the classifier model and classes reference map to be used in the application.

In [31]:
from google.colab import files
import pickle

CLASSIFIER_FILENAME = "classifier.pkl"
CLASSES_FILENAME = "classes.json"

with open(CLASSES_FILENAME, "w") as f:
  json.dump(classes, f)

with open(CLASSIFIER_FILENAME, "wb") as f:
  pickle.dump(model, f)

files.download(CLASSES_FILENAME)
files.download(CLASSIFIER_FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Note**: To load the classifier on its own use the following code. Be wary with pkl files, they can be used to load malware. You should be able to use the code in section 2.1.3 and 2.1.4 without training a new model if you run this. The files must exist in your runtime prior:

In [35]:
import json
import pickle

with open(CLASSIFIER_FILENAME, "rb") as f:
  model = pickle.load(f)

with open("classes.json", "r") as f:
  classes = json.load(f)

#### 2.2 - Translator Training

##### 2.1.1 - DataLoader
This is how we will convert our datasets into a usable format for translation.